In [31]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

In [32]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [33]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [34]:
data[0]

'BV1QB4y1a7r7'

In [35]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1QB4y1a7r7
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/bf62decff70cd983c4bd16ba1409e5f2?auth_key=1720591086-8147ccb18ef341eb92759713e772907f-0-9c7908119dbb1966c2e31290e61fb0e4
Extracted segment: 1Gw411B7S7
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/322899794132025959763cd3dec3f3e6fd6e954b79898d66917?auth_key=1720591098-e2186eba0019413db9c900d8f8a1f1d5-0-d169c00f80559edfa62e8356eb976346
Extracted segment: 1gg4y1R7iu
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/8328784601319701196695ecdcb0a1b7e9a9784183848ae3dcc?auth_key=1720591103-23588f72e068430d8a4c5001863c4430-0-bdf5eb96c6df41206043bdac3f33e664
Extracted segment: 1Nw41127yF
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/319224641128508174578431e064ccd455432384c047cf09af0

In [36]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)(?:\n\n|$)", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [45]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

template_orig = r"""
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一讲解一个题目
    Goals:
    根据给定信息按照要求像老师一样讲解一个题目

    技能
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目
        1. 给出该题目涉及的知识点
        2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤，并对应生成讲解；如果没有原题，则参照相似题的思路和解法给出结果

    第一部分:原题和参考信息
        题目:{question}
        学段：初中，你需要适配初中的知识点，解法和书写表达不超纲
        参考信息:{ref}
    第二部分: 以下列字符串格式输出
        知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ### 板书一:     ### 板书二:       ### 讲解一:       ### 讲解二:      ...
    注意事项:
        1.你擅将利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
        2.你的解题思路、方法、讲解需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
        3.知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。

        4.板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书+讲解单元；讲解是对应板书模块的说明。在板书的最开始按需写出"解："或"证明："，在板书最后写出"答："或"故答案是："。
        5.每个式子要独立成行，不输出连续很长的一个式子。
        6.注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号
        7.禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
        8.如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
        9.讲解和板书一一对应，讲解要重点强调对应板书涉及到的思路和方法，讲解要抓住题目重点，言简意赅，语言精炼。  
        例如:
        "知识点讲解:本题考查的知识点是.....，解题关键点是......。### 板书一: $\because$ $\frac{{x^2-2*x-1}}{{x-1}}=\frac{{(x-1)^2}}{{x-1}}=(x-1)^{{2-1}}=x-1$### 板书二: $\therefore$ $\frac{{x^2-2*x-1}}{{x-1}}*(x-2)=(x-1)(x-2)=x^2-3*x+2$ \n故答案是$x^2-3*x+2$### 讲解一:首先利用完全平方公式将分子化简,随后发现分子分母都是(x-1)的次幂,因此将指数相减即可。### 讲解二:根据上述化简结果,代入原式即可得到最终答案$x^2-3*x+2$"
        10.返回的字段中只能出现知识点讲解、板书N、讲解N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应...不能出任何现其他的字段，在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
        11.板书参考书写范式，你需要根据题型参照范式输出板书：
            {ref_form}
        12.其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题别用括号1234。
            d: 对于所有题目尽可能用解释性的语言进行解答，不到万不得已不用公式直接求解，以最大限度上避免超纲的情况发生。
        13.板书不能超过10条，如果10条写不下，使用别的方法来解题。
"""

In [38]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
某小学一至6年级共780人。在参加数学兴趣小组的学生中，有8/17是6年级学生，有9/23是5年级学生。求该校没有参加数学兴趣小组的学生有几人？

### 讲解：
这里要求没有参加的学生人数。已知总共的人数是780人，我们减去参加数学兴趣小组的人数就可以得到没有参加的人数。接下来我们要求的就是参加数学兴趣小组的人数。已知有8/17是6年级学生，9/23是5年级学生。按照平常的解题思路，只有这两个分类，没有具体的数字，这个题目看似做不了，是求不出来的。

大部分同学分析到这里，认为这个题目缺少条件。我们再仔细看一下，这里到底缺不缺少条件。根据这两个分类，肯定很难直接求出来。这里要结合现实生活，人的个数是整数个的。所以参加兴趣小组的总人数肯定是17和23的公倍数。如果不是的话，6年级和5年级的求出来人数就不是整数个了。

我们得到一个隐藏的已知条件：数学兴趣小组的人数是17和23的公倍数。17和23是互质数，直接相乘得到最小公倍数391人，也就是数学兴趣小组人数最少是391人，或者是391人的倍数。这里有一个上限，总共人数才780人，所以要在小于780的数字中找它的倍数。

391的两倍是782，超过总人数780，所以不符合。只剩下391了。确定数学兴趣小组人数是391人。没有参加的学生人数是780-391=389人。

### 相似题一：
某学校一至6年级共840人。在参加科学兴趣小组的学生中，有7/19是6年级学生，有5/21是5年级学生。求该校没有参加科学兴趣小组的学生有几人？

### 相似题一讲解：
这里要求没有参加的学生人数。已知总共的人数是840人，我们减去参加科学兴趣小组的人数就可以得到没有参加的人数。接下来我们要求的就是参加科学兴趣小组的人数。已知有7/19是6年级学生，5/21是5年级学生。按照平常的解题思路，只有这两个分类，没有具体的数字，这个题目看似做不了，是求不出来的。

大部分同学分析到这里，认为这个题目缺少条件。我们再仔细看一下，这里到底缺不缺少条件。根据这两个分类，肯定很难直接求出来。这里要结合现实生活，人的个数是整数个的。所以参加兴趣小组的总人数肯定是19和21的公倍数。如果不是的话，6年级和5年级的求出来人数就不是整数个了。

我们得到一个隐藏的已知条件：科学兴趣小组的人数是19和21的公倍数。19

In [39]:
raw_problem_answer

['### 原题：\n某小学一至6年级共780人。在参加数学兴趣小组的学生中，有8/17是6年级学生，有9/23是5年级学生。求该校没有参加数学兴趣小组的学生有几人？\n\n### 讲解：\n这里要求没有参加的学生人数。已知总共的人数是780人，我们减去参加数学兴趣小组的人数就可以得到没有参加的人数。接下来我们要求的就是参加数学兴趣小组的人数。已知有8/17是6年级学生，9/23是5年级学生。按照平常的解题思路，只有这两个分类，没有具体的数字，这个题目看似做不了，是求不出来的。\n\n大部分同学分析到这里，认为这个题目缺少条件。我们再仔细看一下，这里到底缺不缺少条件。根据这两个分类，肯定很难直接求出来。这里要结合现实生活，人的个数是整数个的。所以参加兴趣小组的总人数肯定是17和23的公倍数。如果不是的话，6年级和5年级的求出来人数就不是整数个了。\n\n我们得到一个隐藏的已知条件：数学兴趣小组的人数是17和23的公倍数。17和23是互质数，直接相乘得到最小公倍数391人，也就是数学兴趣小组人数最少是391人，或者是391人的倍数。这里有一个上限，总共人数才780人，所以要在小于780的数字中找它的倍数。\n\n391的两倍是782，超过总人数780，所以不符合。只剩下391了。确定数学兴趣小组人数是391人。没有参加的学生人数是780-391=389人。\n\n### 相似题一：\n某学校一至6年级共840人。在参加科学兴趣小组的学生中，有7/19是6年级学生，有5/21是5年级学生。求该校没有参加科学兴趣小组的学生有几人？\n\n### 相似题一讲解：\n这里要求没有参加的学生人数。已知总共的人数是840人，我们减去参加科学兴趣小组的人数就可以得到没有参加的人数。接下来我们要求的就是参加科学兴趣小组的人数。已知有7/19是6年级学生，5/21是5年级学生。按照平常的解题思路，只有这两个分类，没有具体的数字，这个题目看似做不了，是求不出来的。\n\n大部分同学分析到这里，认为这个题目缺少条件。我们再仔细看一下，这里到底缺不缺少条件。根据这两个分类，肯定很难直接求出来。这里要结合现实生活，人的个数是整数个的。所以参加兴趣小组的总人数肯定是19和21的公倍数。如果不是的话，6年级和5年级的求出来人数就不是整数个了。\n\n我们得到一个隐藏的已知条件：科学兴趣小组的人数是

In [40]:
extract_question_explanations(raw_problem_answer)

In [41]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

data

[{'original_question': '某小学一至6年级共780人。在参加数学兴趣小组的学生中，有8/17是6年级学生，有9/23是5年级学生。求该校没有参加数学兴趣小组的学生有几人？',
  'original_explanation': '这里要求没有参加的学生人数。已知总共的人数是780人，我们减去参加数学兴趣小组的人数就可以得到没有参加的人数。接下来我们要求的就是参加数学兴趣小组的人数。已知有8/17是6年级学生，9/23是5年级学生。按照平常的解题思路，只有这两个分类，没有具体的数字，这个题目看似做不了，是求不出来的。\n\n大部分同学分析到这里，认为这个题目缺少条件。我们再仔细看一下，这里到底缺不缺少条件。根据这两个分类，肯定很难直接求出来。这里要结合现实生活，人的个数是整数个的。所以参加兴趣小组的总人数肯定是17和23的公倍数。如果不是的话，6年级和5年级的求出来人数就不是整数个了。\n\n我们得到一个隐藏的已知条件：数学兴趣小组的人数是17和23的公倍数。17和23是互质数，直接相乘得到最小公倍数391人，也就是数学兴趣小组人数最少是391人，或者是391人的倍数。这里有一个上限，总共人数才780人，所以要在小于780的数字中找它的倍数。\n\n391的两倍是782，超过总人数780，所以不符合。只剩下391了。确定数学兴趣小组人数是391人。没有参加的学生人数是780-391=389人。',
  'similar_question_1': '某学校一至6年级共840人。在参加科学兴趣小组的学生中，有7/19是6年级学生，有5/21是5年级学生。求该校没有参加科学兴趣小组的学生有几人？',
  'similar_explanation_1': '这里要求没有参加的学生人数。已知总共的人数是840人，我们减去参加科学兴趣小组的人数就可以得到没有参加的人数。接下来我们要求的就是参加科学兴趣小组的人数。已知有7/19是6年级学生，5/21是5年级学生。按照平常的解题思路，只有这两个分类，没有具体的数字，这个题目看似做不了，是求不出来的。'},
 {'original_question': '一条路修了3/5，距离终点6千米，求这条路的总长。',
  'original_explanation': '这条路修了3/5，距离终点6千米。终点是全长的1/

In [42]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n某小学一至6年级共780人。在参加数学兴趣小组的学生中，有8/17是6年级学生，有9/23是5年级学生。求该校没有参加数学兴趣小组的学生有几人？\n\n讲解:\n这里要求没有参加的学生人数。已知总共的人数是780人，我们减去参加数学兴趣小组的人数就可以得到没有参加的人数。接下来我们要求的就是参加数学兴趣小组的人数。已知有8/17是6年级学生，9/23是5年级学生。按照平常的解题思路，只有这两个分类，没有具体的数字，这个题目看似做不了，是求不出来的。\n\n大部分同学分析到这里，认为这个题目缺少条件。我们再仔细看一下，这里到底缺不缺少条件。根据这两个分类，肯定很难直接求出来。这里要结合现实生活，人的个数是整数个的。所以参加兴趣小组的总人数肯定是17和23的公倍数。如果不是的话，6年级和5年级的求出来人数就不是整数个了。\n\n我们得到一个隐藏的已知条件：数学兴趣小组的人数是17和23的公倍数。17和23是互质数，直接相乘得到最小公倍数391人，也就是数学兴趣小组人数最少是391人，或者是391人的倍数。这里有一个上限，总共人数才780人，所以要在小于780的数字中找它的倍数。\n\n391的两倍是782，超过总人数780，所以不符合。只剩下391了。确定数学兴趣小组人数是391人。没有参加的学生人数是780-391=389人。\n\n题目:\n一条路修了3/5，距离终点6千米，求这条路的总长。\n\n讲解:\n这条路修了3/5，距离终点6千米。终点是全长的1/2。我们知道3/5被分成了两段：1/2和6千米。6千米占全长的1/2。我们可以通过以下步骤求出总长：\n\n1. 设这条路的总长为x千米。\n2. 6千米占全长的1/2，所以全长的一半是6千米，即x/2 = 6千米。\n3. 解这个方程，得到x = 12千米。\n\n所以，这条路的总长是12千米。\n\n题目:\n一本书共有110页，已经看的页数比没看的多1/5，求已经看了多少页？\n\n讲解:\n看到题目当中这里出现了“比”字，后面单位一是没看的页数，给我们的具体量是这一本书的页数，这里是不同意的。其实这道题我们只要转换一下，把这个比值转换成分数，也就是已经看的页数是没看页数的几分之几。根据这句话“已经看的比没看的要多1/5”，这里就是1 + 1/5 = 6/5。已经看的页数是没看页数的6/5。再回

In [43]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [47]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig, max_tokens=2000))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书[二三四五六七八九十]+:[\s\S]*?)*?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl, max_tokens=2000))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是分数和比例的应用，解题关键点是通过分数表示的比例关系求出参加科学兴趣小组的学生人数，再用总人数减去参加科学兴趣小组的学生人数得到未参加的学生人数。

### 板书一:
设参加科学兴趣小组的学生人数为$x$人

$\because$ 6年级学生占科学兴趣小组的$\frac{7}{19}$

$\therefore$ 6年级学生人数为$\frac{7}{19}x$

### 板书二:
$\because$ 5年级学生占科学兴趣小组的$\frac{5}{21}$

$\therefore$ 5年级学生人数为$\frac{5}{21}x$

### 板书三:
$\because$ 6年级学生人数和5年级学生人数之和不能超过总人数

$\therefore$ $\frac{7}{19}x + \frac{5}{21}x \leq 840$

### 板书四:
通分后得：

$\frac{7 \cdot 21 + 5 \cdot 19}{19 \cdot 21}x \leq 840$

$\frac{147 + 95}{399}x \leq 840$

$\frac{242}{399}x \leq 840$

### 板书五:
解得：

$x \leq 840 \cdot \frac{399}{242}$

$x \leq 1386$

### 板书六:
$\therefore$ 参加科学兴趣小组的学生人数最多为1386人

### 板书七:
$\because$ 总人数为840人

$\therefore$ 未参加科学兴趣小组的学生人数为$840 - 1386$

### 板书八:
解得：

未参加科学兴趣小组的学生人数为$-546$

### 讲解一:
首先设参加科学兴趣小组的学生人数为$x$人，根据题意，6年级学生占科学兴趣小组的$\frac{7}{19}$，所以6年级学生人数为$\frac{7}{19}x$。

### 讲解二:
同理，5年级学生占科学兴趣小组的$\frac{5}{21}$，所以5年级学生人数为$\frac{5}{21}x$。

### 讲解三:
根据题意，6年级和5年级学生人数之和不能超过总人数840人，因此有$\frac{7}{19}x + \frac{5}{21}x \leq 840$。

### 

In [51]:
with open("compare_results_p6.md", 'a') as file:
    for i in range(len(questions[:-1])):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')

In [50]:
print(len(refined_explanation))

5


In [32]:
import re

text = """
知识点讲解:本题考查的知识点是因式分解与方程求解，解题关键点是利用平方差公式将原式化简并求解。

###板书一:
解：
$(2a+2b+1)(2a+2b-1)=63$

$设x=2a+2b$

$则(x+1)(x-1)=63$

$x^2-1=63$

$x^2=64$

$x=±8$

$2a+2b=8$

$a+b=4$

$2a+2b=-8$

$a+b=-4$

故答案是$a+b=4$或$a+b=-4$

###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。
"""

pattern = r"###讲解一:[\s\S]*"
match = re.search(pattern, text).group()

if match:
    print(match)
else:
    print("No match found")


###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。

